In [1]:
import os
os.chdir('/content/drive/My Drive/Capstone')

Importing the dataframe

In [ ]:
import pandas as pd

df = pd.read_excel("Final Dataset.xlsx")

So there are 2 ways we will define a transaction:

1. By Purchase Requisition Number
2. By Purchase Order Number

The first way is the best because Purchase Requisition Numbers and Purchase Orders Numbers are in a 1 to many relationship, where there can be 1 purchase requisition number for multiple purchase order numbers. However, in the dataset, there are very few rows with a purchase requisition number so the next best alternative is to define a transaction by purchase order number for these rows.

Hence, we will split the dataset into 2 parts. The first part is for those rows with a purchase requisition number while the second part is for those rows without a purchase requisition number. Then, we will run the apriori algorithm on each separately, then we will simply combine both rules at the end.

For running the apriori algorithm, we will be only generating rules of length 1 only due to space complexity issues. What we meant by length 1 is that for instance, we could have rules like:

(apples, oranges) -> bananas

This means that if a customer buys both apples and oranges, he is likely to buy bananas too. Here, the length is not 1 because the left hand side has 2 items. For an association rule to be length 1, both sides must contain only 1 item.

We will include an association rule if the lift is greater than 1 and the confidence is at least 60%.

# Purchase Recquisition Number

Remove blank recquisition numbers

In [ ]:
pr_df = df[~pd.isna(df['Requisition Number'])]

Retrieve only the final label and pr number for each row

In [ ]:
transaction_ids = []
product_names = []

for index, row in pr_df.iterrows():
  trans_id = row['Requisition Number']
  pn = row['Final Commodity Title']
  transaction_ids.append(trans_id)
  product_names.append(pn)

new_pr_df = pd.DataFrame()
new_pr_df['Transaction Number'] = transaction_ids
new_pr_df['Product Name'] = product_names
new_pr_df['Quantity'] = [1] * new_pr_df.shape[0]

# Purchase Order Number

Find those rows with blank requisition numbers

In [ ]:
po_df = df[pd.isna(df['Requisition Number'])]

Retrieve only the final label and po number for each row

In [ ]:
transaction_ids = []
product_names = []

for index, row in po_df.iterrows():
  trans_id = row['Purchase Order Number']
  pn = row['Final Commodity Title']
  transaction_ids.append(trans_id)
  product_names.append(pn)

new_po_df = pd.DataFrame()
new_po_df['Transaction Number'] = transaction_ids
new_po_df['Product Name'] = product_names
new_po_df['Quantity'] = [1] * new_po_df.shape[0]

# Generating association rules for both PR and PO

## Running Apriori for PR

Generate the basket

In [ ]:
basket = new_pr_df.groupby(['Transaction Number', 'Product Name'])['Quantity'].sum().unstack().fillna(0)

Transform the matrix to a sparse matrix, where the number of columns define the total number of unique items, and each row represents a transaction. Then, for each row, there will only be a 1 for that cell if the transaction contains the item representing that column. For all other columns, the value will be 0.

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

Apriori Algorithm

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

Set the length of the rules to be only 1

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0, use_colnames=True, max_len = 2)

Filter for association rules with lift greater than 1

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

Sort the rules by confidence in descending order

In [ ]:
sub_rules = rules.sort_values(['confidence'], ascending =[False])

Filter for association rules with confidence at least 0.6

In [ ]:
sub_rules = sub_rules[sub_rules['confidence'] >= 0.6]

In [ ]:
pr_rules = sub_rules

## Running Apriori for PO

Generate the basket

In [ ]:
basket = new_p0_df.groupby(['Transaction Number', 'Product Name'])['Quantity'].sum().unstack().fillna(0)

Transform the matrix to a sparse matrix, where the number of columns define the total number of unique items, and each row represents a transaction. Then, for each row, there will only be a 1 for that cell if the transaction contains the item representing that column. For all other columns, the value will be 0.

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

Apriori Algorithm

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

Set the length of the rules to be only 1

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0, use_colnames=True, max_len = 2)

Filter for association rules with lift greater than 1

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

Sort the rules by confidence in descending order

In [ ]:
sub_rules = rules.sort_values(['confidence'], ascending =[False])

Filter for association rules with confidence at least 0.6

In [ ]:
sub_rules = sub_rules[sub_rules['confidence'] >= 0.6]

In [ ]:
po_rules = sub_rules

## Combining the rules

In [ ]:
combined_rules = pd.concat([po_rules, pr_rules])

By default, each item in these 2 columns is in a frozenset. We need to strip it off.

In [ ]:
for index, row in combined_rules.iterrows():
  combined_rules.at[index, 'antecedents'] = list(row['antecedents'])[0]
  combined_rules.at[index, 'consequents'] = list(row['consequents'])[0]

Write to excel file

In [ ]:
combined_rules.to_excel('Association Rules combined.xlsx')

# Prepare data for Tableau

We are almost there. So now, we have to prepare the association rules for tableau so that we can create the association dashboard on Tableau.

## Assign IDs to each unique product

Import excel file

In [2]:
import pandas as pd

df = pd.read_excel('Association Rules combined.xlsx')

Get the unique products from the first 2 columns.

In [4]:
import numpy as np

unique_products = np.unique(df[['From', 'To']].values)

In [ ]:
import random

sampled_list = random.sample(list(range(1, 1208)), 1207)


Assign an index number to each unique product.

In [8]:
unique_dct = {}
idx = 0
for i in unique_products:
  unique_dct[i] = sampled_list[idx]
  idx += 1

Create 2 columns, representing the id of the from product and the to product.

In [9]:
froms = []
tos = []
startings = []
endings = []
confidences = []

for index, row in df.iterrows():
  curr_from = row['From']
  curr_to = row['To']
  curr_confidence = row['Confidence']
  froms.append(curr_from)
  tos.append(curr_to)
  startings.append(unique_dct[curr_from])
  endings.append(unique_dct[curr_to])
  confidences.append(curr_confidence)

data = pd.DataFrame()
data['From'] = froms
data['To'] = tos
data['Starting'] = startings
data['Ending'] = endings
data['Value'] = confidences
data.head()

,From,To,Starting,Ending,Value
0,Split washer,Needlenose pliers,560,1088,1.0
1,Mechanical connectors,Lab coats,167,425,1.0
2,Motor brush,Bolt cutters,143,929,1.0
3,Vial closure cap or seal or stopper,Medical ultrasound or doppler or echo transduc...,960,591,1.0
4,Split washer,Grinding and sanding and polishing equipment a...,560,4,1.0


## Retrieve UNSPSC Codes

Now, we need to retrieve the UNSPSC codes for the products in each row. We will do this with the final dataset of y1 + 3 months of data.

In [10]:
original_df = pd.read_excel('Final Dataset.xlsx')

original_df.head()

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,Department Name,Supplier Code,Supplier Name,Supplier Qualifications,Supplier Zip Code,CalCard,Item Name,Item Description,Quantity,Unit Price,Total Price,Classification Codes,Normalized UNSPSC (Editted),Commodity Title (Editted),Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location,REMOVE AMERISOURCE,Unnamed: 34,Final Commodity Title,Final UNSPSC Code,Final Class Label,Final Class UNSPSC,Final Family Label,Final Family UNSPSC,Final Segment Label,Final Segment UNSPSC
0,2013-04-05 00:00:00,2013-02-28 00:00:00,2012-2013,NaN,4500171679,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,"Corrections and Rehabilitation, Department of",50613,US Control Group Inc,CA-DVBE CA-MB CA-SB CA-SBE CDVBE,90213,NO,BOILER HOUSE SUPPLIES,"O-RINGS, AIR FILTER, HOSE ASSY, SEAT BELTS, OI...",1.0,8586.00,8586.00,15101701\n15121501\n40161505,40101851.0,Boiler parts and accessories,15101701.0,#2 Heating fuel oil,15101700.0,Fuel Oils,15100000.0,Fuels,15000000.0,Fuels and Fuel Additives and Lubricants and An...,"90213\n(34.080294, -118.404679)",NaN,NaN,Boiler parts and accessories,40101851,Heating equipment and parts and accessories,40101800,Heating and ventilation and air circulation,40100000,Distribution and Conditioning Systems and Equi...,40000000
1,2012-10-08 00:00:00,2012-10-02 00:00:00,2012-2013,NaN,CC12-016,NaN,NON-IT Goods,NaN,Fair and Reasonable,NaN,Student Aid Commission,1028097,Staples,NaN,93422,YES,CC12-016,Universal wireless clicker for slide show pres...,1.0,0.00,0.00,43223102\n43222640,45111700.0,Audio presentation and composing equipment and...,43223102.0,2G GSM wireless access network equipment and c...,43223100.0,Digital mobile network infrastructure equipmen...,43220000.0,Data Voice or Multimedia Network Equipment or ...,43000000.0,Information Technology Broadcasting and Teleco...,"93422\n(35.475579, -120.679205)",NaN,NaN,Audio presentation and composing equipment and...,45111700,Audio presentation and composing equipment and...,45111700,Audio and visual presentation and composing eq...,45110000,Printing and Photographic and Audio and Visual...,45000000
2,2012-09-11 00:00:00,2010-08-31 00:00:00,2012-2013,NaN,59A0734-A01,NaN,NON-IT Services,Personal Services,Services are specifically exempt by statute,NaN,"Transportation, Department of",1745285,John Boda,NaN,NaN,NO,59A0734,Seismic Advisory Consultation for two year ser...,1.0,40000.00,40000.00,71112303\n71151105,71112328.0,Borehole seismic monitoring acquisition services,71112303.0,2d/ 3d/ 4d seismic data processing services,71112300.0,Seismic services,71110000.0,Oil and gas exploration services,71000000.0,Mining and oil and gas services,NaN,NaN,NaN,Seismic services,71112300,Seismic services,71112300,Oil and gas exploration services,71110000,Mining and oil and gas services,71000000
3,2012-07-12 00:00:00,2012-07-12 00:00:00,2012-2013,NaN,12-3-1029-001,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,"Rehabilitation, Department of",1176080,A-1 Distributing,NaN,95828,NO,Outsider Vending Machine,Outsider Vending Machine,1.0,10174.25,10174.25,48111104,48110000.0,Vending machines,48111104.0,A la carte foods vending machines,48111100.0,Piece and part vending machines,48110000.0,Vending machines,48000000.0,Service Industry Machinery and Equipment and S...,"95828\n(38.484836, -121.399547)",NaN,NaN,Vending machines,48110000,Vending machines,48110000,Vending machines,48110000,Service Industry Machinery and Equipment and S...,48000000
4,2012-08-22 00:00:00,2012-08-22 00:00:00,2012-2013,NaN,12-3-898.1-019,NaN,NON-IT Goods,NaN,SB/DVBE Option,NaN,"Rehabilitation, Department of",1477860,"GVR Services, Inc.",CA-SB,95822,NO,Vending Machines,Vending Machines,1.0,43567.01,43567.01,48111104\n48111101,48110000.0,Vending machines,48111104.0,A la carte foods vending machines,48111100.0,Piece and part

Create 4 dictionaries here since the UNSPSC is a hierachical structure of 4 levels.

In [11]:
comm_dct = {}
class_dct = {}
fam_dct = {}
seg_dct = {}

for index, row in original_df.iterrows():
  comm = row['Final Commodity Title']
  comm_code = row['Final UNSPSC Code']
  clas = row['Final Class Label']
  class_code = row['Final Class UNSPSC']
  fam = row['Final Family Label']
  fam_code = row['Final Family UNSPSC']
  seg = row['Final Segment Label']
  seg_code = row['Final Segment UNSPSC']
  if comm not in comm_dct:
    comm_dct[comm] = comm_code
  if clas not in class_dct:
    class_dct[clas] = class_code
  if fam not in fam_dct:
    fam_dct[fam] = fam_code
  if seg not in seg_dct:
    seg_dct[seg] = seg_code

Write a function to search for the code in each dictionary sequentially

In [12]:
def search_code(label):
  if label in comm_dct:
    return comm_dct[label]
  elif label in class_dct:
    return class_dct[label]
  elif label in fam_dct:
    return fam_dct[label]
  elif label in seg_dct:
    return seg_dct[label]

Retrieve the UNSPSC Codes

In [13]:
from_codes = []
to_codes = []

for index, row in data.iterrows():
  curr_from = row['From']
  curr_to = row['To']
  from_code = search_code(curr_from)
  to_code = search_code(curr_to)
  from_codes.append(from_code)
  to_codes.append(to_code)

data['From UNSPSC'] = from_codes
data['To UNSPSC'] = to_codes

data.head()

,From,To,Starting,Ending,Value,From UNSPSC,To UNSPSC
0,Split washer,Needlenose pliers,560,1088,1.0,31161837,27112108
1,Mechanical connectors,Lab coats,167,425,1.0,39121408,46181532
2,Motor brush,Bolt cutters,143,929,1.0,26101404,27111512
3,Vial closure cap or seal or stopper,Medical ultrasound or doppler or echo transduc...,960,591,1.0,41121821,42201711
4,Split washer,Grinding and sanding and polishing equipment a...,560,4,1.0,31161837,23131500


## Tabulate the total price for the target product in each association rule

Due to the nature of the dataset, we are unable to get the unit cost of each product. Hence, we can only calculate the total cost of all rows containing that product.

Filter rows with total price below 0.

In [14]:
sub_original_df = original_df[original_df['Total Price'] >= 0]

Again, based on the hierachical structure of UNSPSC, we will calculate the average total cost per transaction based on the UNSPSC code.

In [15]:
comm_res = sub_original_df.groupby('Final UNSPSC Code')['Total Price'].agg('sum')
class_res = sub_original_df.groupby('Final Class UNSPSC')['Total Price'].agg('sum')
fam_res = sub_original_df.groupby('Final Family UNSPSC')['Total Price'].agg('sum')
seg_res = sub_original_df.groupby('Final Segment UNSPSC')['Total Price'].agg('sum')

Assign the total cost to each rule based on the target product only 

In [16]:
total_prices = []

for index, row in data.iterrows():
  to_code = row['To UNSPSC']
  if to_code in comm_res:
    to_price = comm_res.loc[to_code,]
  elif to_code in class_res:
    to_price = class_res.loc[to_code,]
  elif to_code in fam_res:
    to_price = fam_res.loc[to_code,]
  elif to_code in seg_res:
    to_price = seg_res.loc[to_code,]
  else:
    to_price = 0
  total_prices.append(to_price)

data['Total Cost for Item'] = total_prices
data.head()

,From,To,Starting,Ending,Value,From UNSPSC,To UNSPSC,Total Cost for Item
0,Split washer,Needlenose pliers,560,1088,1.0,31161837,27112108,413.47
1,Mechanical connectors,Lab coats,167,425,1.0,39121408,46181532,42112.71
2,Motor brush,Bolt cutters,143,929,1.0,26101404,27111512,410.10
3,Vial closure cap or seal or stopper,Medical ultrasound or doppler or echo transduc...,960,591,1.0,41121821,42201711,70256.77
4,Split washer,Grinding and sanding and polishing equipment a...,560,4,1.0,31161837,23131500,3923.17


## Tabulate the total number of transactions for the target product in each association rule

Due to the nature of the dataset, we are unable to get the exact quantity in each transaction since there can be bulk orders. Hence, we can only calculate the total number of transactions containing the specific product.

Filter rows with quantity below 0.

In [17]:
sub_original_df = original_df[original_df['Quantity'] >= 0]

Again, based on the hierachical structure of UNSPSC, we will calculate the total number of transactions based on the UNSPSC code.

In [18]:
comm_res = sub_original_df.groupby('Final UNSPSC Code').size()
class_res = sub_original_df.groupby('Final Class UNSPSC').size()
fam_res = sub_original_df.groupby('Final Family UNSPSC').size()
seg_res = sub_original_df.groupby('Final Segment UNSPSC').size()

Assign the total number of transactions to each rule based on the target product only 

In [19]:
total_quantites = []

for index, row in data.iterrows():
  to_code = row['To UNSPSC']
  if to_code in comm_res:
    to_qty = comm_res.loc[to_code,]
  elif to_code in class_res:
    to_qty = class_res.loc[to_code,]
  elif to_code in fam_res:
    to_qty = fam_res.loc[to_code,]
  elif to_code in seg_res:
    to_qty = seg_res.loc[to_code,]
  else:
    to_qty = 0
  total_quantites.append(to_qty)

data['Total Number of Transactions'] = total_quantites
data.head()

,From,To,Starting,Ending,Value,From UNSPSC,To UNSPSC,Total Cost for Item,Total Number of Transactions
0,Split washer,Needlenose pliers,560,1088,1.0,31161837,27112108,413.47,5
1,Mechanical connectors,Lab coats,167,425,1.0,39121408,46181532,42112.71,39
2,Motor brush,Bolt cutters,143,929,1.0,26101404,27111512,410.10,3
3,Vial closure cap or seal or stopper,Medical ultrasound or doppler or echo transduc...,960,591,1.0,41121821,42201711,70256.77,5
4,Split washer,Grinding and sanding and polishing equipment a...,560,4,1.0,31161837,23131500,3923.17,4


We are done! Output the dataframe to an excel file and we can use it to create our association dashboard on Tableau.

In [20]:
data.to_excel('Association Labelled Data.xlsx')

It was a fruitful project. Thank you for staying through and reading all 4 notebooks.